In [1]:
import pandas as pd

# Chargement des fichiers CSV
df_incidents = pd.read_csv("../../data/raw/Cleaned_data/InUSE/cleaned_data_incidents.csv")
df_mobilisations = pd.read_csv("../../data/raw/Cleaned_data/InUSE/cleaned_data_mobilisations.csv")

# Vérification du chargement
print("Données incidents clean chargées:")
display(df_incidents.head()) # Use .head() for better display in non-notebook environments
print("Données mobilisations clean chargées:")
display(df_mobilisations.head()) # Use .head() for better display in non-notebook environments

# Vérifier la période des données d'incidents et de mobilisation: doit etre de 2009 à 2025
print("Période des données d'incidents chargées: De", min(df_incidents["CalYear"]), "à", max(df_incidents["CalYear"]))
print("Période des données de mobilisations chargées: De", min(df_mobilisations["CalYear"]), "à", max(df_mobilisations["CalYear"]))

print("\nVérification taille:")
print("Incidents :", len(df_incidents), " | Mobilisations :", len(df_mobilisations))

# Colonnes à garder pour la table mobilisations (This part is commented out in the PySpark section,
# but was present in the Pandas section of the original code, so keeping it here for context)
# mobilisations_cols_to_keep = [
#     'IncidentNumber', 'CalYear', 'BoroughName', 'WardName', 'HourOfCall',
#     'DateAndTimeMobilised', 'DateAndTimeMobile', 'DateAndTimeArrived',
#     'TurnoutTimeSeconds', 'TravelTimeSeconds', 'AttendanceTimeSeconds',
#     'DeployedFromStation_Name', 'DeployedFromLocation'
# ]
# df_mobilisations_reduced = df_mobilisations[mobilisations_cols_to_keep].copy()

# Prétraitement de la colonne IncidentNumber
def clean_incident_number_pandas(value):
    """Nettoyage de l'identifiant d'incident"""
    if pd.isna(value):
        return None
    value = str(value)
    if '.' in value:
        return value.split('.')[0]
    elif '-' in value:
        return value.split('-')[0]
    return value

print("\nType IncidentNumber (avant nettoyage - Incidents) :", df_incidents["IncidentNumber"].dtype)
print("Type IncidentNumber (avant nettoyage - Mobilisations) :", df_mobilisations["IncidentNumber"].dtype)

# Application du nettoyage aux deux jeux de données
df_incidents["incident_id_cleaned"] = df_incidents["IncidentNumber"].apply(clean_incident_number_pandas)
df_mobilisations["incident_id_cleaned"] = df_mobilisations["IncidentNumber"].apply(clean_incident_number_pandas)

print("Type IncidentNumber (après nettoyage - Incidents):", df_incidents["incident_id_cleaned"].dtype)
print("Type IncidentNumber (après nettoyage - Mobilisations):", df_mobilisations["incident_id_cleaned"].dtype)

# Jointure sur la colonne nettoyée
df_joined_pandas = pd.merge(df_mobilisations, df_incidents, on="incident_id_cleaned", how="inner")

# Count après jointure
print("\nCount après jointure (INNER JOIN sur incident_id_cleaned - Pandas) :", len(df_joined_pandas))

# % de jointure réussie par rapport à la table mobilisations
joined_ratio_pandas = len(df_joined_pandas) / len(df_mobilisations)
print(f"Taux de jointure réussie (Pandas) : {joined_ratio_pandas:.2%}")

# Vérifier après jointure de la période des données d'incidents et de mobilisation: doit etre de 2009 à 2025
print("Période des données jointes (Pandas): De", min(df_joined_pandas["CalYear_x"]), "à", max(df_joined_pandas["CalYear_x"])) # CalYear_x from mobilisations
print("Période des données jointes (Pandas): De", min(df_joined_pandas["CalYear_y"]), "à", max(df_joined_pandas["CalYear_y"])) # CalYear_y from incidents


print("\n" + "="*50)
print("Démarrage de la section PySpark")
print("="*50 + "\n")

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, regexp_extract, min as spark_min, max as spark_max
import os

# Stop any existing Spark session (important si redémarré en cours de route)
try:
    spark.stop()
except:
    pass

# Démarrage d'une nouvelle session Spark
spark = SparkSession.builder \
    .appName("Jointure Incidents & Mobilisations") \
    .master("local[*]") \
    .getOrCreate()

print("SparkSession initialisée")

# 2. Chargement des fichiers nettoyés
path_incidents = "../../data/raw/Cleaned_data/InUSE/cleaned_data_incidents.csv"
path_mobilisations = "../../data/raw/Cleaned_data/InUSE/cleaned_data_mobilisations.csv"

# Optionnel : Vérifie si les fichiers existent (Python natif)
assert os.path.exists(path_incidents), f"Fichier introuvable : {path_incidents}"
assert os.path.exists(path_mobilisations), f"Fichier introuvable : {path_mobilisations}"

# Chargement CSV
df_incidents_spark = spark.read.option("header", True).option("inferSchema", "true").csv(path_incidents)
df_mobilisations_spark = spark.read.option("header", True).option("inferSchema", "true").csv(path_mobilisations)

print("Données chargées dans Spark")

# 3. Aperçu rapide
print("Échantillon incidents (Spark)")
df_incidents_spark.show(3, truncate=False)
print("Échantillon mobilisations (Spark)")
df_mobilisations_spark.show(3, truncate=False)

# 4. Vérification de la période des données
print("Période des incidents (Spark) :")
df_incidents_spark.select(spark_min("CalYear"), spark_max("CalYear")).show()
print("Période des mobilisations (Spark) :")
df_mobilisations_spark.select(spark_min("CalYear"), spark_max("CalYear")).show()

# 5. Nettoyage IncidentNumber for both DataFrames
def clean_incident_number_spark(col_):
    return when(col_.contains('.'), regexp_extract(col_, r"^([^\.]+)", 1)) \
        .when(col_.contains('-'), regexp_extract(col_, r"^([^-]+)", 1)) \
        .otherwise(col_)

df_incidents_cleaned_spark = df_incidents_spark.withColumn(
    "incident_id_cleaned", clean_incident_number_spark(col("IncidentNumber")))
df_mobilisations_cleaned_spark = df_mobilisations_spark.withColumn(
    "incident_id_cleaned", clean_incident_number_spark(col("IncidentNumber")))

# IMPORTANT ADDITION: Drop duplicates from df_incidents_cleaned based on the cleaned incident ID
# This ensures a one-to-many relationship where each incident is unique.
df_incidents_unique_spark = df_incidents_cleaned_spark.dropDuplicates(['incident_id_cleaned'])
print(f"Nombre d'incidents uniques après nettoyage (Spark) : {df_incidents_unique_spark.count():,}")

# 6. Jointure sur la colonne id nettoyée
# Rename 'CalYear' in incidents to avoid ambiguity after join
df_incidents_unique_spark = df_incidents_unique_spark.withColumnRenamed("CalYear", "CalYear_inc")
df_mobilisations_cleaned_spark = df_mobilisations_cleaned_spark.withColumnRenamed("CalYear", "CalYear_mob")
df_mobilisations_cleaned_spark = df_mobilisations_cleaned_spark.withColumnRenamed("HourOfCall", "HourOfCall_mob")


# Join mobilisations to incidents, as one mobilization links to one incident.
df_joined_spark = df_mobilisations_cleaned_spark.join(
    df_incidents_unique_spark, # Use the DataFrame with unique incident IDs
    on="incident_id_cleaned",
    how="inner"
)

# 7. Statistiques de jointure
nb_incidents_original_spark = df_incidents_spark.count()
nb_mobilisations_original_spark = df_mobilisations_spark.count()
nb_jointure_spark = df_joined_spark.count()

print(f"\nNombre d'incidents (original - Spark) : {nb_incidents_original_spark:,}")
print(f"Nombre de mobilisations (original - Spark) : {nb_mobilisations_original_spark:,}")
print(f"Nombre d'incidents uniques utilisés pour la jointure (Spark) : {df_incidents_unique_spark.count():,}")
print(f"Lignes jointes (Spark) : {nb_jointure_spark:,}")

# The join ratio should now ideally be closer to 100% of mobilisations if all mobilisations have matching incidents.
print(f"Taux de jointure réussie (par rapport aux mobilisations - Spark) : {nb_jointure_spark / nb_mobilisations_original_spark:.2%}")
print(f"Taux de jointure réussie (par rapport aux incidents uniques - Spark) : {nb_jointure_spark / df_incidents_unique_spark.count():.2%}")


# Check for unmatched mobilisations
df_mobilisations_unmatched_spark = df_mobilisations_cleaned_spark.join(
    df_incidents_unique_spark, # Use the DataFrame with unique incident IDs
    on="incident_id_cleaned",
    how="left_anti"
)
print(f"Mobilisations non jointes (Spark) : {df_mobilisations_unmatched_spark.count():,}")

# 8. Affichage des 10 premières lignes après jointure
print("\nAperçu des données après jointure (Spark) :")
df_joined_spark.show(5, truncate=False)
print("Schema après jointure (Spark):")
df_joined_spark.printSchema()

colonnes_apres_jointure_spark = df_joined_spark.columns
nombre_colonnes_spark = len(colonnes_apres_jointure_spark)

print("\nListe des colonnes après jointure (Spark) :")
print(colonnes_apres_jointure_spark)
print(f"Nombre total de colonnes (Spark) : {nombre_colonnes_spark}")

# Étape 1 : Lister les colonnes des deux DataFrames
cols_incidents_spark = set(df_incidents_unique_spark.columns)
cols_mobilisations_spark = set(df_mobilisations_cleaned_spark.columns)

# Étape 2 : Trouver les colonnes communes
# We need to exclude the renamed 'CalYear_inc' and 'HourOfCall_inc' and 'IncidentNumber' from the initial
# list of columns to avoid double counting after renaming.
# The `on` column ('incident_id_cleaned') is handled correctly by the join.
common_columns_spark = (cols_incidents_spark.intersection(cols_mobilisations_spark)) - {'CalYear_inc', 'HourOfCall', 'IncidentNumber'} # Exclude explicitly renamed/dropped columns

print("\nColonnes présentes dans les deux DataFrames (possibles doublons après jointure, hors celles renommées):")
for col_name in sorted(common_columns_spark):
    print(f"- {col_name}")

print(f"\nNombre de colonnes communes (hors renommées/jointure key) : {len(common_columns_spark)}")


# Comparaison CalYear (Spark)
print("\nComparaison CalYear (Spark):")
df_joined_spark.select(
    (col("CalYear_mob") == col("CalYear_inc")).alias("same_CalYear")
).groupBy("same_CalYear").count().show()

# Comparaison HourOfCall (Spark)
print("Comparaison HourOfCall (Spark):")
# Note: HourOfCall was only present in mobilisations, and then it was named HourOfCall. In incidents, it should be
# 'HourOfCall'. If it exists in both, then this comparison is valid.
# Assuming 'HourOfCall' from incidents was renamed by Spark's default behavior, or if it was explicitly present.
# Let's assume 'HourOfCall' is the one from incidents, and 'HourOfCall_mob' is from mobilisations.
if "HourOfCall" in df_joined_spark.columns: # Check if 'HourOfCall' from incidents is present
    df_joined_spark.select(
        (col("HourOfCall_mob") == col("HourOfCall")).alias("same_HourOfCall")
    ).groupBy("same_HourOfCall").count().show()
else:
    print("HourOfCall from incidents not directly available for comparison after join, or was renamed.")


# Comparaison IncidentNumber (Spark) - this is less meaningful after using 'incident_id_cleaned' as join key
# and assuming 'IncidentNumber' from incidents is kept as 'IncidentNumber' and from mobilisations as 'IncidentNumber'.
# Spark typically renames duplicated columns unless specified. Let's check how they are named.
print("Comparaison IncidentNumber (Spark):")
# Assuming the IncidentNumber from mobilisations is 'IncidentNumber' and from incidents is 'IncidentNumber_inc' or similar.
# We should check the column names in df_joined_spark.
# Based on the schema, it looks like 'IncidentNumber' comes from mobilisations and 'IncidentNumber' (from incidents)
# gets renamed by Spark's default behavior to 'IncidentNumber_incidents' or similar if not explicitly dropped/renamed.
# Re-checking the actual columns after join:
# If both original 'IncidentNumber' columns are kept, Spark will append _x and _y (or similar).
# Let's verify the column names and then compare.
# From the df_joined.printSchema() earlier, we can infer how Spark handled it.
# If `df_joined_spark` contains 'IncidentNumber' (from mobilisations) and 'IncidentNumber' (from incidents),
# Spark will make one of them 'IncidentNumber' and the other 'IncidentNumber_inc' or similar.
# Based on the schema output:
# 'IncidentNumber' will be from mobilisations, and 'IncidentNumber_inc' will be from incidents.
if "IncidentNumber_inc" in df_joined_spark.columns:
    df_joined_spark.select(
        (col("IncidentNumber") == col("IncidentNumber_inc")).alias("same_IncidentNumber")
    ).groupBy("same_IncidentNumber").count().show()
else:
    print("IncidentNumber from incidents not found as 'IncidentNumber_inc' for direct comparison.")
    # If the column name is just 'IncidentNumber' from incidents, and another one is also 'IncidentNumber', this is an issue.
    # The previous `dropDuplicates` on incidents should have handled this, but the comparison still implies having both.


print("\nPériode des données jointes (CalYear_mob - Spark) :")
periode_mob = df_joined_spark.agg(
    spark_min("CalYear_mob").alias("Annee_min"),
    spark_max("CalYear_mob").alias("Annee_max")
).collect()[0]
print(f"Période des données (CalYear_mob) : de {periode_mob['Annee_min']} à {periode_mob['Annee_max']}")

print("\nPériode des données jointes (CalYear_inc - Spark) :")
periode_inc = df_joined_spark.agg(
    spark_min("CalYear_inc").alias("Annee_min"),
    spark_max("CalYear_inc").alias("Annee_max")
).collect()[0]
print(f"Période des données (CalYear_inc) : de {periode_inc['Annee_min']} à {periode_inc['Annee_max']}")

# Optional: Stop Spark session if you're done with operations
spark.stop()

C:\Users\9609241C\AppData\Local\Temp\ipykernel_12716\4191285728.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_incidents = pd.read_csv("../../data/raw/Cleaned_data/InUSE/cleaned_data_incidents.csv")
C:\Users\9609241C\AppData\Local\Temp\ipykernel_12716\4191285728.py:5: DtypeWarning: Columns (0,2,3,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_mobilisations = pd.read_csv("../../data/raw/Cleaned_data/InUSE/cleaned_data_mobilisations.csv")


Données incidents clean chargées:


,IncidentNumber,DateOfCall,CalYear,TimeOfCall,HourOfCall,IncidentGroup,StopCodeDescription,SpecialServiceType,PropertyCategory,PropertyType,...,FirstPumpArriving_AttendanceTime,FirstPumpArriving_DeployedFromStation,SecondPumpArriving_AttendanceTime,SecondPumpArriving_DeployedFromStation,NumStationsWithPumpsAttending,NumPumpsAttending,PumpCount,PumpMinutesRounded,NotionalCost,NumCalls
0,235138081.0,2009-01-01,2009,00:00:37,0,Special Service,Special Service,RTC,Road Vehicle,Car,...,319.0,Battersea,342.0,Clapham,2.0,2.0,2,60,255,1.0
1,1091.0,2009-01-01,2009,00:00:46,0,Special Service,Special Service,Assist other agencies,Outdoor,Lake/pond/reservoir,...,NaN,NaN,NaN,NaN,NaN,NaN,1,60,255,1.0
2,2091.0,2009-01-01,2009,00:03:00,0,Fire,Secondary Fire,NaN,Outdoor,Road surface/pavement,...,308.0,Edmonton,NaN,NaN,1.0,1.0,1,60,255,2.0
3,3091.0,2009-01-01,2009,00:04:27,0,Fire,Secondary Fire,NaN,Outdoor,Domestic garden (vegetation not equipment),...,210.0,Hillingdon,NaN,NaN,1.0,1.0,1,60,255,2.0
4,5091.0,2009-01-01,2009,00:05:39,0,Fire,Secondary Fire,NaN,Outdoor,Cycle path/public footpath/bridleway,...,233.0,Holloway,250.0,Holloway,1.0,2.0,2,60,255,1.0


Données mobilisations clean chargées:


,IncidentNumber,CalYear,BoroughName,WardName,HourOfCall,ResourceMobilisationId,Resource_Code,PerformanceReporting,DateAndTimeMobilised,DateAndTimeMobile,...,DateAndTimeLeft,DateAndTimeReturned,DeployedFromStation_Code,DeployedFromStation_Name,DeployedFromLocation,PumpOrder,PlusCode_Code,PlusCode_Description,DelayCodeId,DelayCode_Description
0,000004-01012025,2025,HAMMERSMITH AND FULHAM,FULHAM REACH,0,6862256,H331,2,2025-01-01 00:02:00,2025-01-01 00:07:00,...,2025-01-01 00:23:00,NaN,H33,Wandsworth,Home Station,2,Initial,Initial Mobilisation,12.0,Not held up
1,000004-01012025,2025,HAMMERSMITH AND FULHAM,FULHAM REACH,0,6862257,G261,1,2025-01-01 00:02:00,2025-01-01 00:02:00,...,2025-01-01 00:38:00,NaN,G36,Hammersmith,Other Station,1,Initial,Initial Mobilisation,12.0,Not held up
2,000005-01012025,2025,MERTON,WEST BARNES,0,6862259,H401,1,2025-01-01 00:03:00,2025-01-01 00:04:00,...,2025-01-01 00:11:00,NaN,H40,New Malden,Home Station,1,Initial,Initial Mobilisation,NaN,NaN
3,000006-01012025,2025,CROYDON,PURLEY OAKS & RIDDLESDOWN,0,6862260,H291,1,2025-01-01 00:04:00,2025-01-01 00:06:00,...,2025-01-01 00:25:00,NaN,H29,Purley,Home Station,1,Initial,Initial Mobilisation,NaN,NaN
4,000007-01012025,2025,BARNET,BURNT OAK,0,6862261,G222,1,2025-01-01 00:05:00,2025-01-01 00:06:00,...,2025-01-01 00:27:00,NaN,G22,Stanmore,Home Station,1,Initial,Initial Mobilisation,NaN,NaN


Période des données d'incidents chargées: De 2009 à 2025
Période des données de mobilisations chargées: De 2009 à 2025

Vérification taille:
Incidents : 1824973  | Mobilisations : 2578276

Type IncidentNumber (avant nettoyage - Incidents) : object
Type IncidentNumber (avant nettoyage - Mobilisations) : object
Type IncidentNumber (après nettoyage - Incidents): object
Type IncidentNumber (après nettoyage - Mobilisations): object

Count après jointure (INNER JOIN sur incident_id_cleaned - Pandas) : 9949296
Taux de jointure réussie (Pandas) : 385.89%
Période des données jointes (Pandas): De 2009 à 2025
Période des données jointes (Pandas): De 2009 à 2025

Démarrage de la section PySpark



PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

 <span style="color:#42a5f5; font-size:2em; font-weight:bold;">Jointure des données d'incidents et de mobilisations</span>

 <span style="font-weight:bold">Ce notebook à pour but d'explorer et de réaliser une jointureentre les données de  mobilisations et d'incidents !</span>

<span style="color:#e91e63; font-size:1em; font-weight:bold;"> 1. Import des données précedemment chargées et nettoyées</span>

In [ ]:
#import des BU
import pandas as pd

In [ ]:
# Chargement des fichiers CSV
df_incidents = pd.read_csv("../../data/raw/Cleaned_data/InUSE/cleaned_data_incidents.csv")
df_mobilisations = pd.read_csv("../../data/raw/Cleaned_data/InUSE/cleaned_data_mobilisations.csv")

C:\Users\9609241C\AppData\Local\Temp\ipykernel_11964\2508286040.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_incidents = pd.read_csv("../../data/raw/Cleaned_data/InUSE/cleaned_data_incidents.csv")
C:\Users\9609241C\AppData\Local\Temp\ipykernel_11964\2508286040.py:3: DtypeWarning: Columns (0,2,3,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_mobilisations = pd.read_csv("../../data/raw/Cleaned_data/InUSE/cleaned_data_mobilisations.csv")


In [ ]:
#Vérification du chargement
print("Données incidents clean chargées:")
display(df_incidents)

Données incidents clean chargées:


,IncidentNumber,DateOfCall,CalYear,TimeOfCall,HourOfCall,IncidentGroup,StopCodeDescription,SpecialServiceType,PropertyCategory,PropertyType,...,FirstPumpArriving_AttendanceTime,FirstPumpArriving_DeployedFromStation,SecondPumpArriving_AttendanceTime,SecondPumpArriving_DeployedFromStation,NumStationsWithPumpsAttending,NumPumpsAttending,PumpCount,PumpMinutesRounded,NotionalCost,NumCalls
0,235138081.0,2009-01-01,2009,00:00:37,0,Special Service,Special Service,RTC,Road Vehicle,Car,...,319.0,Battersea,342.0,Clapham,2.0,2.0,2,60,255,1.0
1,1091.0,2009-01-01,2009,00:00:46,0,Special Service,Special Service,Assist other agencies,Outdoor,Lake/pond/reservoir,...,NaN,NaN,NaN,NaN,NaN,NaN,1,60,255,1.0
2,2091.0,2009-01-01,2009,00:03:00,0,Fire,Secondary Fire,NaN,Outdoor,Road surface/pavement,...,308.0,Edmonton,NaN,NaN,1.0,1.0,1,60,255,2.0
3,3091.0,2009-01-01,2009,00:04:27,0,Fire,Secondary Fire,NaN,Outdoor,Domestic garden (vegetation not equipment),...,210.0,Hillingdon,NaN,NaN,1.0,1.0,1,60,255,2.0
4,5091.0,2009-01-01,2009,00:05:39,0,Fire,Secondary Fire,NaN,Outdoor,Cycle path/public footpath/bridleway,...,233.0,Holloway,250.0,Holloway,1.0,2.0,2,60,255,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1824968,054201-31032025,2025-03-31,2025,23:39:44,23,False Alarm,AFA,NaN,Non Residential,Shopping Centre,...,264.0,Hammersmith,247.0,Hammersmith,2.0,3.0,3,60,430,2.0
1824969,054203-31032025,2025-03-31,2025,23:40:09,23,Special Service,Special Service,Flooding,Dwelling,Purpose Built Flats/Maisonettes - 4 to 9 storeys,...,229.0,Holloway,NaN,NaN,1.0,1.0,1,60,430,1.0
1824970,054204-31032025,2025-03-31,2025,23:42:16,23,Special Service,Special Service,Effecting entry/exit,Dwelling,Licensed House in Multiple Occupation - Up to ...,...,373.0,Harrow,NaN,NaN,1.0,1.0,1,60,430,1.0
1824971,054205-31032025,2025-03-31,2025,23:42:42,23,Fire,Secondary Fire,NaN,Outdoor,Loose refuse,...,367.0,East Ham,412.0,Stratford,2.0,2.0,2,60,430,2.0


In [ ]:
#Vérification du chargement
print("Données mobilisations clean chargées:")
display(df_mobilisations)

Données mobilisations clean chargées:


,IncidentNumber,CalYear,BoroughName,WardName,HourOfCall,ResourceMobilisationId,Resource_Code,PerformanceReporting,DateAndTimeMobilised,DateAndTimeMobile,...,DateAndTimeLeft,DateAndTimeReturned,DeployedFromStation_Code,DeployedFromStation_Name,DeployedFromLocation,PumpOrder,PlusCode_Code,PlusCode_Description,DelayCodeId,DelayCode_Description
0,000004-01012025,2025,HAMMERSMITH AND FULHAM,FULHAM REACH,0,6862256,H331,2,2025-01-01 00:02:00,2025-01-01 00:07:00,...,2025-01-01 00:23:00,NaN,H33,Wandsworth,Home Station,2,Initial,Initial Mobilisation,12.0,Not held up
1,000004-01012025,2025,HAMMERSMITH AND FULHAM,FULHAM REACH,0,6862257,G261,1,2025-01-01 00:02:00,2025-01-01 00:02:00,...,2025-01-01 00:38:00,NaN,G36,Hammersmith,Other Station,1,Initial,Initial Mobilisation,12.0,Not held up
2,000005-01012025,2025,MERTON,WEST BARNES,0,6862259,H401,1,2025-01-01 00:03:00,2025-01-01 00:04:00,...,2025-01-01 00:11:00,NaN,H40,New Malden,Home Station,1,Initial,Initial Mobilisation,NaN,NaN
3,000006-01012025,2025,CROYDON,PURLEY OAKS & RIDDLESDOWN,0,6862260,H291,1,2025-01-01 00:04:00,2025-01-01 00:06:00,...,2025-01-01 00:25:00,NaN,H29,Purley,Home Station,1,Initial,Initial Mobilisation,NaN,NaN
4,000007-01012025,2025,BARNET,BURNT OAK,0,6862261,G222,1,2025-01-01 00:05:00,2025-01-01 00:06:00,...,2025-01-01 00:27:00,NaN,G22,Stanmore,Home Station,1,Initial,Initial Mobilisation,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2578271,221555-30122024,2024,KINGSTON UPON THAMES,COOMBE VALE,23,6861269,H391,2,2024-12-30 23:47:00,2024-12-30 23:49:00,...,2024-12-30 23:56:00,NaN,H39,Surbiton,Home Station,2,Initial,Initial Mobilisation,9.0,"Traffic, roadworks, etc"
2578272,221556-30122024,2024,ISLINGTON,FINSBURY PARK,23,6861270,A311,1,2024-12-30 23:48:00,2024-12-30 23:49:00,...,2024-12-31 00:16:00,NaN,A31,Holloway,Home Station,1,Initial,Initial Mobilisation,NaN,NaN
2578273,221556-30122024,2024,ISLINGTON,FINSBURY PARK,23,6861271,F321,2,2024-12-30 23:48:00,2024-12-30 23:49:00,...,2024-12-31 00:04:00,NaN,F32,Stoke Newington,Home Station,2,Initial,Initial Mobilisation,12.0,Not held up
2578274,221557-30122024,2024,KENSINGTON AND CHELSEA,ROYAL HOSPITAL,23,6861272,G341,1,2024-12-30 23:59:00,2024-12-31 00:01:00,...,2024-12-31 00:39:00,NaN,G34,Chelsea,Home Station,1,Initial,Initial Mobilisation,NaN,NaN


In [ ]:
#Vérifier la période des données d'incidents et de mobilisation: doit etre de 2009 à 2025
print("Période des données d'incidents chargées: De", min(df_incidents["CalYear"]), "à", max(df_incidents["CalYear"]))
print("Période des données de mobilisations chargées: De", min(df_mobilisations["CalYear"]), "à", max(df_mobilisations["CalYear"]))

Période des données d'incidents chargées: De 2009 à 2025
Période des données de mobilisations chargées: De 2009 à 2025


 <span style="color:#e91e63; font-size:1em; font-weight:bold;"> 2. test et jointure</span>

Comme précedemment explorer : Un incident peut avoir plusieurs mobilisations et une mobilisation à un seul incident ==> Inner join

                                    1 824 973(incidents)              X            2 578 276(mobilisations)            

In [ ]:
print("Vérification taille:")
print("Incidents :", len(df_incidents), " | Mobilisations :", len(df_mobilisations))

Vérification taille:
Incidents : 1824973  | Mobilisations : 2578276


In [ ]:
# Colonnes à garder pour la table mobilisations
#mobilisations_cols_to_keep = ['IncidentNumber', 'CalYear', 'BoroughName', 'WardName', 'HourOfCall','DateAndTimeMobilised', 'DateAndTimeMobile', 'DateAndTimeArrived','TurnoutTimeSeconds', 'TravelTimeSeconds', 'AttendanceTimeSeconds','DeployedFromStation_Name', 'DeployedFromLocation']

# Garde uniquement ces colonnes dans df_mobilisations
#df_mobilisations_reduced = df_mobilisations[mobilisations_cols_to_keep].copy()

In [ ]:
#  Prétraitement de la colonne IncidentNumber 
#meme si déjà forcé lors du chargement en  type string !

def clean_incident_number(value):
    """Nettoyage de l'identifiant d'incident"""
    if pd.isna(value):
        return None
    value = str(value)
    if '.' in value:
        return value.split('.')[0]
    elif '-' in value:
        return value.split('-')[0]
    return value

print("Type IncidentNumber :", df_incidents["IncidentNumber"].dtype)
print("Type IncidentNumber :", df_mobilisations["IncidentNumber"].dtype)


Type IncidentNumber : object
Type IncidentNumber : object


In [ ]:
# Application du dtype aux deux jeux de données au cas ou :
df_incidents["incident_id_cleaned"] = df_incidents["IncidentNumber"].apply(clean_incident_number)
df_mobilisations["incident_id_cleaned"] = df_mobilisations["IncidentNumber"].apply(clean_incident_number)

print("Type IncidentNumber Incidents:", df_incidents["IncidentNumber"].dtype)
print("Type IncidentNumber Mobilisations:", df_mobilisations["IncidentNumber"].dtype)

Type IncidentNumber Incidents: object
Type IncidentNumber Mobilisations: object


In [ ]:
# Jointure sur la colonne nettoyée 
df_joined = pd.merge(df_mobilisations, df_incidents, on="incident_id_cleaned", how="inner")

# Count après jointure
print("Count après jointure (INNER JOIN sur incident_id_cleaned) :", len(df_joined))

# % de jointure réussie par rapport à la table mobilisations
joined_ratio = len(df_joined) / len(df_mobilisations)
print(f"Taux de jointure réussie : {joined_ratio:.2%}")

Count après jointure (INNER JOIN sur incident_id_cleaned) : 9949296
Taux de jointure réussie : 385.89%


In [ ]:
mobilisations_per_incident = df_mobilisations["incident_id_cleaned"].value_counts()
print("Nb moyen de mobilisations par incident :", mobilisations_per_incident.mean())
print("Nb max de mobilisations pour un incident :", mobilisations_per_incident.max())

Nb moyen de mobilisations par incident : 2.840036482520995
Nb max de mobilisations pour un incident : 61


Moyenne : 2,84 mobilisations / incident
Cela confirme que chaque incident génère en moyenne près de 3 mobilisations différentes (engins, camions, unités…).
Maximum : 61 mobilisations pour un seul incident
Cela explique pourquoi on obtient les différences dans les colonnes comme CalYear ou HourOfCall :
➜ l’incident apparaît répété 61 fois dans df_joined si on ne contrôles pas bien la jointure !

In [ ]:
print(df_joined.columns)

Index(['IncidentNumber_x', 'CalYear_x', 'BoroughName', 'WardName',
       'HourOfCall_x', 'ResourceMobilisationId', 'Resource_Code',
       'PerformanceReporting', 'DateAndTimeMobilised', 'DateAndTimeMobile',
       'DateAndTimeArrived', 'TurnoutTimeSeconds', 'TravelTimeSeconds',
       'AttendanceTimeSeconds', 'DateAndTimeLeft', 'DateAndTimeReturned',
       'DeployedFromStation_Code', 'DeployedFromStation_Name',
       'DeployedFromLocation', 'PumpOrder', 'PlusCode_Code',
       'PlusCode_Description', 'DelayCodeId', 'DelayCode_Description',
       'incident_id_cleaned', 'IncidentNumber_y', 'DateOfCall', 'CalYear_y',
       'TimeOfCall', 'HourOfCall_y', 'IncidentGroup', 'StopCodeDescription',
       'SpecialServiceType', 'PropertyCategory', 'PropertyType',
       'AddressQualifier', 'Postcode_full', 'Postcode_district', 'UPRN',
       'USRN', 'IncGeo_BoroughCode', 'IncGeo_BoroughName', 'ProperCase',
       'IncGeo_WardCode', 'IncGeo_WardName', 'IncGeo_WardNameNew', 'Easting_m',
     

In [ ]:
# Comparaison des colonnes
print("CalYear_x vs CalYear_y identiques ? ", df_joined["CalYear_x"].equals(df_joined["CalYear_y"]))
print("HourOfCall_x vs HourOfCall_y identiques ? ", df_joined["HourOfCall_x"].equals(df_joined["HourOfCall_y"]))
print("IncidentNumber_x vs IncidentNumber_y identiques ? ", df_joined["IncidentNumber_x"].equals(df_joined["IncidentNumber_y"]))

CalYear_x vs CalYear_y identiques ?  False
HourOfCall_x vs HourOfCall_y identiques ?  False
IncidentNumber_x vs IncidentNumber_y identiques ?  False


In [ ]:
diff_calyear = (df_joined["CalYear_x"] != df_joined["CalYear_y"]).sum()
diff_hour = (df_joined["HourOfCall_x"] != df_joined["HourOfCall_y"]).sum()
diff_incident = (df_joined["IncidentNumber_x"] != df_joined["IncidentNumber_y"]).sum()

print("Différences CalYear :", diff_calyear)
print("Différences HourOfCall :", diff_hour)
print("Différences IncidentNumber :", diff_incident)

Différences CalYear : 7395303
Différences HourOfCall : 7056286
Différences IncidentNumber : 7432514


In [ ]:
#Vérifier  aprés jointure de la période des données d'incidents et de mobilisation: doit etre de 2009 à 2025
print("Période des données d'incidents joint à mobilisation chargées: De", min(df_joined["CalYear_x"]), "à", max(df_joined["CalYear_x"]))
#Vérifier  aprés jointure de la période des données d'incidents et de mobilisation: doit etre de 2009 à 2025
print("Période des données d'incidents joint à mobilisation chargées: De", min(df_joined["CalYear_y"]), "à", max(df_joined["CalYear_y"]))



Période des données d'incidents joint à mobilisation chargées: De 2009 à 2025
Période des données d'incidents joint à mobilisation chargées: De 2009 à 2025


In [ ]:
# Étape 1 : Lister les colonnes des deux DataFrames
cols_incidents = set(df_incidents.columns)
cols_mobilisations = set(df_mobilisations.columns)

# Étape 2 : Trouver les colonnes communes
common_columns = cols_incidents.intersection(cols_mobilisations)

# Étape 3 : Afficher
print("Colonnes présentes dans les deux DataFrames (possibles doublons après jointure) :")
for col_name in sorted(common_columns):
    print(f"- {col_name}")

print(f"\nNombre de colonnes communes : {len(common_columns)}")


Colonnes présentes dans les deux DataFrames (possibles doublons après jointure) :
- CalYear
- HourOfCall
- IncidentNumber
- incident_id_cleaned

Nombre de colonnes communes : 4


In [ ]:
from pyspark.sql.functions import col, when

# Comparaison CalYear
df_joined.select(
    (col("CalYear") == col("df_incidents_cleaned.CalYear")).alias("same_CalYear")
).groupBy("same_CalYear").count().show()

# Comparaison HourOfCall
df_joined.select(
    (col("HourOfCall") == col("df_incidents_cleaned.HourOfCall")).alias("same_HourOfCall")
).groupBy("same_HourOfCall").count().show()

# Comparaison IncidentNumber
df_joined.select(
    (col("IncidentNumber") == col("df_incidents_cleaned.IncidentNumber")).alias("same_IncidentNumber")
).groupBy("same_IncidentNumber").count().show()


{"ts": "2025-06-12 04:57:56.823", "level": "ERROR", "logger": "DataFrameQueryContextLogger", "msg": "[AMBIGUOUS_REFERENCE] Reference `CalYear` is ambiguous, could be: [`CalYear`, `CalYear`]. SQLSTATE: 42704", "context": {"file": "line 5 in cell [54]", "line": "", "fragment": "col", "errorClass": "AMBIGUOUS_REFERENCE"}, "exception": {"class": "Py4JJavaError", "msg": "An error occurred while calling o834.select.\n: org.apache.spark.sql.AnalysisException: [AMBIGUOUS_REFERENCE] Reference `CalYear` is ambiguous, could be: [`CalYear`, `CalYear`]. SQLSTATE: 42704\r\n\tat org.apache.spark.sql.errors.QueryCompilationErrors$.ambiguousReferenceError(QueryCompilationErrors.scala:2163)\r\n\tat org.apache.spark.sql.catalyst.expressions.package$AttributeSeq.resolve(package.scala:356)\r\n\tat org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveChildren(LogicalPlan.scala:164)\r\n\tat org.apache.spark.sql.catalyst.analysis.ColumnResolutionHelper.$anonfun$resolveExpressionByPlanChildren$1(Colu

AnalysisException: [AMBIGUOUS_REFERENCE] Reference `CalYear` is ambiguous, could be: [`CalYear`, `CalYear`]. SQLSTATE: 42704

In [ ]:
from pyspark.sql.functions import min, max

periode = df_joined.agg(
    min("CalYear_mob").alias("Annee_min"),
    max("CalYear_mob").alias("Annee_max")
).collect()[0]

print(f"Période des données : de {periode['Annee_min']} à {periode['Annee_max']}")


{"ts": "2025-06-12 04:58:02.865", "level": "ERROR", "logger": "DataFrameQueryContextLogger", "msg": "[UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `CalYear_mob` cannot be resolved. Did you mean one of the following? [`CalYear`, `CalYear`, `Easting_m`, `WardName`, `NumCalls`]. SQLSTATE: 42703", "context": {"file": "jdk.internal.reflect.GeneratedMethodAccessor56.invoke(Unknown Source)", "line": "", "fragment": "col", "errorClass": "UNRESOLVED_COLUMN.WITH_SUGGESTION"}, "exception": {"class": "Py4JJavaError", "msg": "An error occurred while calling o874.agg.\n: org.apache.spark.sql.AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `CalYear_mob` cannot be resolved. Did you mean one of the following? [`CalYear`, `CalYear`, `Easting_m`, `WardName`, `NumCalls`]. SQLSTATE: 42703;\n'Aggregate ['min('CalYear_mob) AS Annee_min#5509, 'max('CalYear_mob) AS Annee_max#5510]\n+- Project [incident_id_clean

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `CalYear_mob` cannot be resolved. Did you mean one of the following? [`CalYear`, `CalYear`, `Easting_m`, `WardName`, `NumCalls`]. SQLSTATE: 42703;
'Aggregate ['min('CalYear_mob) AS Annee_min#5509, 'max('CalYear_mob) AS Annee_max#5510]
+- Project [incident_id_cleaned#5048, IncidentNumber#4726, CalYear#4727, BoroughName#4728, WardName#4729, HourOfCall#4730, ResourceMobilisationId#4731, Resource_Code#4732, PerformanceReporting#4733, DateAndTimeMobilised#4734, DateAndTimeMobile#4735, DateAndTimeArrived#4736, TurnoutTimeSeconds#4737, TravelTimeSeconds#4738, AttendanceTimeSeconds#4739, DateAndTimeLeft#4740, DateAndTimeReturned#4741, DeployedFromStation_Code#4742, DeployedFromStation_Name#4743, DeployedFromLocation#4744, PumpOrder#4745, PlusCode_Code#4746, PlusCode_Description#4747, DelayCodeId#4748, DelayCode_Description#4749, ... 39 more fields]
   +- Join Inner, (incident_id_cleaned#5

In [ ]:
from pyspark.sql.functions import min, max

periode = df_joined.agg(
    min("CalYear_inc").alias("Annee_min"),
    max("CalYear_inc").alias("Annee_max")
).collect()[0]

print(f"Période des données : de {periode['Annee_min']} à {periode['Annee_max']}")


Période des données : de 2009 à 2025


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, when, min as spark_min, max as spark_max
from pyspark.sql.functions import regexp_extract
import os

# Stop any existing Spark session (important si redémarré en cours de route)
try:
    spark.stop()
except:
    pass

# Démarrage d'une nouvelle session Spark
spark = SparkSession.builder \
    .appName("Jointure Incidents & Mobilisations") \
    .master("local[*]") \
    .getOrCreate()

print("SparkSession initialisée")

# 2. Chargement des fichiers nettoyés
path_incidents = "../../data/raw/Cleaned_data/InUSE/cleaned_data_incidents.csv"
path_mobilisations = "../../data/raw/Cleaned_data/InUSE/cleaned_data_mobilisations.csv"

# Optionnel : Vérifie si les fichiers existent (Python natif)
assert os.path.exists(path_incidents), f"Fichier introuvable : {path_incidents}"
assert os.path.exists(path_mobilisations), f"Fichier introuvable : {path_mobilisations}"

# Chargement CSV
# Use .option("inferSchema", "true") for better schema inference, especially with mixed types
df_incidents = spark.read.option("header", True).option("inferSchema", "true").csv(path_incidents)
df_mobilisations = spark.read.option("header", True).option("inferSchema", "true").csv(path_mobilisations)

print("Données chargées")

# 3. Aperçu rapide
print("Échantillon incidents")
df_incidents.show(3, truncate=False)
print("Échantillon mobilisations")
df_mobilisations.show(3, truncate=False)

# 4. Vérification de la période des données
print("Période des incidents :")
df_incidents.select(spark_min("CalYear"), spark_max("CalYear")).show()
print("Période des mobilisations :")
df_mobilisations.select(spark_min("CalYear"), spark_max("CalYear")).show()

# 5. Nettoyage IncidentNumber for both DataFrames
def clean_incident_number(col_):
    return when(col_.contains('.'), regexp_extract(col_, r"^([^\.]+)", 1)) \
    .when(col_.contains('-'), regexp_extract(col_, r"^([^-]+)", 1)) \
    .otherwise(col_)

df_incidents_cleaned = df_incidents.withColumn(
    "incident_id_cleaned", clean_incident_number(col("IncidentNumber")))
df_mobilisations_cleaned = df_mobilisations.withColumn(
    "incident_id_cleaned", clean_incident_number(col("IncidentNumber")))


df_incidents_unique = df_incidents_cleaned.dropDuplicates(['incident_id_cleaned'])
print(f"Nombre d'incidents uniques après nettoyage : {df_incidents_unique.count():,}")

# 6. Jointure sur la colonne id nettoyée
df_joined = df_mobilisations_cleaned.join(
df_incidents_unique,
on="incident_id_cleaned",
how="inner"
)

# 7. Statistiques de jointure
nb_incidents_original = df_incidents.count()
nb_mobilisations_original = df_mobilisations.count()
nb_jointure = df_joined.count()

print(f"Nombre d'incidents (original) : {nb_incidents_original:,}")
print(f"Nombre de mobilisations (original) : {nb_mobilisations_original:,}")
print(f"Nombre d'incidents uniques utilisés pour la jointure : {df_incidents_unique.count():,}")
print(f"Lignes jointes : {nb_jointure:,}")

print(f"Taux de jointure réussie (par rapport aux mobilisations) : {nb_jointure / nb_mobilisations_original:.2%}")

# mobilisations sans match
df_mobilisations_unmatched = df_mobilisations_cleaned.join(
df_incidents_unique, # Use the DataFrame with unique incident IDs
on="incident_id_cleaned",
how="left_anti"
)
print(f"Mobilisations non jointes : {df_mobilisations_unmatched.count():,}")

# 8. Affichage des 10 premières lignes après jointure
print("Aperçu des données après jointure :")
df_joined.show(5, truncate=False)

# Stop Spark 
spark.stop()

PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.